<a href="https://colab.research.google.com/github/mpnsk/ivy_seminar/blob/main/ivy_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ivy
!wget https://www.statlearning.com/s/Advertising.csv
!pip install dm-haiku

--2023-08-10 06:23:50--  https://www.statlearning.com/s/Advertising.csv
Resolving www.statlearning.com (www.statlearning.com)... 198.185.159.144, 198.49.23.145, 198.185.159.145, ...
Connecting to www.statlearning.com (www.statlearning.com)|198.185.159.144|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://static1.squarespace.com/static/5ff2adbe3fe4fe33db902812/t/5fffe03b4091076ff5b30c72/1610604603901/Advertising.csv [following]
--2023-08-10 06:23:51--  https://static1.squarespace.com/static/5ff2adbe3fe4fe33db902812/t/5fffe03b4091076ff5b30c72/1610604603901/Advertising.csv
Resolving static1.squarespace.com (static1.squarespace.com)... 151.101.0.238, 151.101.64.238, 151.101.128.238, ...
Connecting to static1.squarespace.com (static1.squarespace.com)|151.101.0.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4555 (4.4K) [text/csv]
Saving to: ‘Advertising.csv’

Advertising.csv     100%[===================>]   4.45K  --.-KB/s  

In [7]:
import ivy

# my_backend = "torch"
# my_backend = "tensorflow"
my_backend = "jax"
if my_backend == "jax":
  import jax
  jax.config.update('jax_enable_x64', True)

ivy.set_backend(my_backend)

class Net(ivy.Module):
    def __init__(self):
        self.fc1 = ivy.Linear(1, 1)
        super().__init__()

    def _forward(self, x):
        x = self.fc1(x)
        return x


arg = ivy.random_normal(shape=(1,1))

model = Net()
model = ivy.compile(model, to=my_backend, args=(arg,))

import pandas as pd
df = pd.read_csv ("Advertising.csv")
x = ivy.array(df['radio'].values.astype('float32')).reshape((200 ,1 ))
y = ivy.array(df['sales'].values.astype('float32')).reshape((200 ,1 ))


def loss_fn(v):
    y_new = model(x, v=v)
    return ivy.mean((y - y_new) ** 2)

lr = 0.001
# for e in range(15000):
for e in range(200):
    loss, grads = ivy.execute_with_gradients(loss_fn, model.v)
    model.v = model.v - grads * lr
    if e % 10 == 0:
        print("epoch:", e, "loss: ", loss)


/usr/local/lib/python3.10/dist-packages/ivy/compiler/compiler.py:93: FutureWarning: NaT.freq is deprecated and will be removed in a future version.
  return _compile(
/usr/local/lib/python3.10/dist-packages/ivy/compiler/compiler.py:93: FutureWarning: _AXIS_NUMBERS has been deprecated.
  return _compile(
/usr/local/lib/python3.10/dist-packages/ivy/compiler/compiler.py:93: FutureWarning: _AXIS_NAMES has been deprecated.
  return _compile(
/usr/local/lib/python3.10/dist-packages/ivy/compiler/compiler.py:93: FutureWarning: Index.asi8 is deprecated and will be removed in a future version.
  return _compile(
/usr/local/lib/python3.10/dist-packages/ivy/compiler/compiler.py:93: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  return _compile(
/usr/local/lib/python3.10/dist-packages/ivy/compiler/compiler.py:93: FutureWarning: RangeIndex._start is deprecated and will be removed in a future version. Use RangeIndex.start inst

epoch: 0 loss:  ivy.array(1238.8036, dev=gpu:0)
epoch: 10 loss:  ivy.array(42.610046, dev=gpu:0)
epoch: 20 loss:  ivy.array(42.326317, dev=gpu:0)
epoch: 30 loss:  ivy.array(42.048557, dev=gpu:0)
epoch: 40 loss:  ivy.array(41.773983, dev=gpu:0)
epoch: 50 loss:  ivy.array(41.502552, dev=gpu:0)
epoch: 60 loss:  ivy.array(41.234234, dev=gpu:0)
epoch: 70 loss:  ivy.array(40.968987, dev=gpu:0)
epoch: 80 loss:  ivy.array(40.70679, dev=gpu:0)
epoch: 90 loss:  ivy.array(40.447594, dev=gpu:0)
epoch: 100 loss:  ivy.array(40.19137, dev=gpu:0)
epoch: 110 loss:  ivy.array(39.938076, dev=gpu:0)
epoch: 120 loss:  ivy.array(39.68769, dev=gpu:0)
epoch: 130 loss:  ivy.array(39.440174, dev=gpu:0)
epoch: 140 loss:  ivy.array(39.19549, dev=gpu:0)
epoch: 150 loss:  ivy.array(38.953613, dev=gpu:0)
epoch: 160 loss:  ivy.array(38.714516, dev=gpu:0)
epoch: 170 loss:  ivy.array(38.478153, dev=gpu:0)
epoch: 180 loss:  ivy.array(38.244495, dev=gpu:0)
epoch: 190 loss:  ivy.array(38.013523, dev=gpu:0)


In [18]:
x_new = ivy.array([23.0]).astype(ivy.float64)
y_new = model(x_new)
print('predict', y_new)
# print('w: ', linear_layer.v.w, 'b: ', linear_layer.v.b)
print(model.v)

predict tensor([-1.4502e+232], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
{
    fc1: {
        b: tensor([-1.9268e+229], device=cuda:0, dtype=torch.float64, 
           grad_fn=<SubBackward0>),
        w: tensor([[-6.2967e+230]], device=cuda:0, dtype=torch.float64, 
           grad_fn=<SubBackward0>)
    }
}
